Now that we have our hatsTrees that have the interesting physical variables calculated, let's do some analysis with them. To combine our MC background samples, we will need to compute weights for them. Fortunately, we have these defined in python `.ini` files.

`.ini` files are a standard format for python configuration files. They have a simple syntax and are quite flexible -- this is another example of where python can help us from falling in the trap of re-inventing the wheel by writing custom code for every simple task, like parsing text files.

Let's take a look at `hatsConfig.ini`.

In [ ]:
!cat hatsConfig.ini

In [ ]:
from ConfigParser import RawConfigParser
config = RawConfigParser()   
config.optionxform = str       # Last two lines are done because ConfigParser will not preserve case
config.read("hatsConfig.ini")

Here is a nifty way to create a dict of the cross sections and number of events:

In [ ]:
crossSections = dict([sample, float(xsec)] for sample, xsec in config.items('hatsXsects'))
nProcessed    = dict([sample, int(nPro)] for sample, nPro in config.items('hatsNprocessed'))

from pprint import pprint
print "cross sections:" 
pprint(crossSections)
print "number of events processed:"
pprint(nProcessed)

python `dict`s are extremely useful, because we can give descriptive names to the data they hold. Let's use our dicts to calculate the weights for our MC background samples.

In [ ]:
weights = {}
luminosity = 1.42    # This is just an example value
for sample in crossSections.keys():
    weights[sample] = luminosity * crossSections[sample]/nProcessed[sample]
pprint(weights)

We can make dicts to hold TChains of all our data, and then draw them with weights:

In [ ]:
import ROOT as r
from os import listdir, path, popen
hatsTreesDir = "root://cmseos.fnal.gov//store/user/hats/PyRoot/2017/hatsDijetTrees"
hatsChains = {}
for sample in crossSections.keys():
    sampleDir = hatsTreesDir + "/hatsTrees_" + sample
    chain = r.TChain('hatsDijets')
    for hatsFile in filter(None,popen("xrdfs root://cmseos.fnal.gov/ ls -u "+sampleDir).read().split('\n')):
        chain.Add(hatsFile)
    hatsChains[sample] = chain
pprint(hatsChains)

Now we can try to make weighted histograms of all the MC backgrounds using TChain.Draw(), and put them into a stackplot. Here we run into a classic pyROOT gotcha: it's not easy to prevent root from garbage collecting your histograms. It's best to keep them in an array that isn't within the scope of a loop.

In [ ]:
hists = {}
for sample in crossSections.keys():
    varNames=[]
    for var in hatsChains[sample].GetListOfBranches():
        varNames.append(var.GetName())
    for varName in varNames:
        histLabel = "%s_%s" % (varName, sample)
        hists[histLabel]=r.TH1F(histLabel, histLabel, 100, 0, 0)
        hatsChains[sample].Draw("%s>>%s" % (varName, histLabel))

pprint(hists)

Now that we've made histograms of all our variables in all our samples, we can put together stack plots of them all. We will leave that as an exercise to work on for the rest of the HATS. The histograms are organized in a dictionary that you should be able to navigate easily using their keys.

In [ ]:
canvas = r.TCanvas()
hists["cosThetaStar_QCD_HT1000to1500"].Draw()
canvas.Draw()